In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from random import sample


In [2]:
def readCsv(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment2\Datasets\DS' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None,
                 comment='#', sep=',')
    
    return df

In [3]:
def createData():
    df = readCsv('1_Cov.csv')
    meanPositive = readCsv('1_m_1.csv').dropna(axis=1, how='all').as_matrix()
    meanNegative = readCsv('1_m_0.csv').dropna(axis=1, how='all').as_matrix()
    meanPositive = meanPositive[0,:]
    meanNegative = meanNegative[0,:]
    
    cov = df.dropna(axis=1, how='all').as_matrix()
    
    distributionPositive = np.random.multivariate_normal(meanPositive, cov, 2000)
    distributionNegative = np.random.multivariate_normal(meanNegative, cov, 2000)
    
    distributionPositive = np.c_[distributionPositive, np.ones(distributionPositive.shape[0])]
    distributionNegative = np.c_[distributionNegative, np.zeros(distributionNegative.shape[0])]
#     print('Positive', distributionPositive)
#     print('Negative', distributionNegative)
    distribution = np.concatenate((distributionPositive, distributionNegative), axis=0)
    np.random.shuffle(distribution)
    distributionTest = distribution[0:1200]
    distributionTrain = distribution[1200:] 
    print(distribution)
    print(distributionTest)
    print(distributionTrain)
    print('original', distribution.shape)
    print('train', distributionTrain.shape)
    print('test', distributionTest.shape)
    return (distribution, distributionTest, distributionTrain)


In [5]:
def toCsv(fileName, npArray):
    
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment2\Datasets\DS' + fileName
    np.savetxt(fullFileName, npArray, delimiter=',')


In [6]:
def initializeDS1():
    fullDistribution, distributionTest, distributionTrain = createData()
    toCsv('1_train.csv', distributionTrain)
    toCsv('1_test.csv', distributionTest)
    toCsv('1.csv', fullDistribution)
initializeDS1()


[[-1.2610321  -0.02021385  0.38602819 ...,  1.08067437  0.27332503  1.        ]
 [ 4.90165259  4.25032949  5.17282906 ...,  3.50091679  5.46227538  0.        ]
 [ 0.07024861  1.37195268  0.61756295 ...,  0.46289041 -0.65990557  1.        ]
 ..., 
 [-1.76468317 -1.58817167 -4.03249017 ..., -2.63190315  0.74487943  0.        ]
 [-1.18851705 -0.92035588 -0.92336192 ..., -1.78221879  0.1074397   0.        ]
 [ 0.61565258  0.74100025  3.03642729 ...,  1.6065076   1.58360587  0.        ]]
[[-1.2610321  -0.02021385  0.38602819 ...,  1.08067437  0.27332503  1.        ]
 [ 4.90165259  4.25032949  5.17282906 ...,  3.50091679  5.46227538  0.        ]
 [ 0.07024861  1.37195268  0.61756295 ...,  0.46289041 -0.65990557  1.        ]
 ..., 
 [ 4.76569914  2.47416548  2.50101246 ...,  2.14858651  2.1930285   0.        ]
 [-3.56683359 -0.80591817 -0.78378181 ..., -0.61234415 -0.67620237  1.        ]
 [-0.42568736 -1.76242478  0.52895824 ...,  0.14104568 -2.42013696  0.        ]]
[[ 2.94279471  3.6383705

In [124]:
def visualizeStuff():
    mean = [10, 10, 10]
    cov = [[1, 0, 0], [0, 1, 0], [0, 0, 1]] 
    x, y, z = np.random.multivariate_normal(mean, cov, 10000).T

    fig = plt.figure(figsize=(20,8))
    ax = fig.add_subplot(111, projection='3d')

    plt.scatter(x, y, z, c='red')
    plt.axis('equal')
    plt.show()
# visualizeStuff()

In [48]:
def runLDA():
    dfTrain = readCsv('1_train.csv')
    N1, N2, pi, S, S1, S2, U1, U2 = np.zeros(8)
    Y = dfTrain[20].as_matrix()
    
    print(Y)
    N1 = np.count_nonzero(dfTrain[20].as_matrix())
    N2 = 2800 - N1
    pi = N1/(N1+N2)
    dfTrain = dfTrain.iloc[:,0:20]
    print(dfTrain)
#     for i in range(0, len(dfTrain)):
#         print(dfTrain.iloc[i,20])
#         U1 += dfTrain.iloc[i,20]*
runLDA()


[ 1.  1.  0. ...,  0.  0.  0.]
            0          1         2         3         4         5         6   \
0     2.942795   3.638371  4.515129  1.635249  1.809344  5.117315  3.016894   
1     0.789092  -2.872807  0.018848 -0.462386 -0.717092 -0.121865  0.980037   
2     1.509422   0.681433 -0.633665 -0.067692  0.717304 -0.916498  1.373753   
3     2.896100   2.875818  1.341019  4.968281  3.802280  1.255279  1.556910   
4    -0.715349  -1.040282 -1.365613  0.046887 -1.229878 -0.966895  0.764011   
5     2.861976   1.971477  0.960188  4.407544  5.352976 -0.178799  3.519170   
6     2.951837   6.572051  3.569875  2.104681  2.358884  4.995112  1.729433   
7     4.524464   5.416465  6.099523  2.258304  4.617884  4.712204  2.964046   
8     3.242962   1.622316  2.121090  3.292105  2.315143  4.432647  1.639804   
9     2.251539   4.942593  3.190409  4.096935  3.467200  2.589286  3.222833   
10    2.827657   6.402725  4.579642  3.411185  4.129089  4.990948  4.992226   
11    4.312875   4.08